In [1]:
from operator import length_hint

import torch
import torch.nn as nn
import torch.optim as optim

import pprint

from A2.SP_24_CS224N_PyTorch_Tutorial import batch_size, shuffle

pp = pprint.PrettyPrinter()

In [2]:
# Our raw data, which consists of sentences
corpus = [
          "We always come to Paris",
          "The professor is from Australia",
          "I live in Stanford",
          "He comes from Taiwan",
          "The capital of Turkey is Ankara"
         ]

In [3]:
### this is the preprocessing steps that we will take for our data
def preprocess_sentence(sentence):
    return sentence.lower().split()

train_sentences = [preprocess_sentence(sent) for sent in corpus]
train_sentences

[['we', 'always', 'come', 'to', 'paris'],
 ['the', 'professor', 'is', 'from', 'australia'],
 ['i', 'live', 'in', 'stanford'],
 ['he', 'comes', 'from', 'taiwan'],
 ['the', 'capital', 'of', 'turkey', 'is', 'ankara']]

In [4]:
locations = set(['paris', 'australia', 'stanford', 'taiwan', 'turkey', 'ankara'])

train_labels = [[1 if word in locations else 0 for word in sent] for sent in train_sentences]
train_labels


[[0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1]]

In [5]:
vocabulary = set(w for s in train_sentences for w in s)
vocabulary

{'always',
 'ankara',
 'australia',
 'capital',
 'come',
 'comes',
 'from',
 'he',
 'i',
 'in',
 'is',
 'live',
 'of',
 'paris',
 'professor',
 'stanford',
 'taiwan',
 'the',
 'to',
 'turkey',
 'we'}

In [6]:
vocabulary.add("<unk>")
vocabulary.add("<pad>")

In [7]:
def pad_window(sentence, window_size, pad_token="<pad>"):
    window = [pad_token] * window_size
    return window + sentence + window

window_size = 2
pad_window(train_sentences[0], window_size=window_size)

['<pad>', '<pad>', 'we', 'always', 'come', 'to', 'paris', '<pad>', '<pad>']

In [8]:
# We are just converting our vocabularly to a list to be able to index into it
# Sorting is not necessary, we sort to show an ordered word_to_ind dictionary
# That being said, we will see that having the index for the padding token
# be 0 is convenient as some PyTorch functions use it as a default value
# such as nn.utils.rnn.pad_sequence, which we will cover in a bit
ix_into_words = sorted(list(vocabulary))
word_into_ix = {word: ind for ind, word in enumerate(ix_into_words)}
word_into_ix

{'<pad>': 0,
 '<unk>': 1,
 'always': 2,
 'ankara': 3,
 'australia': 4,
 'capital': 5,
 'come': 6,
 'comes': 7,
 'from': 8,
 'he': 9,
 'i': 10,
 'in': 11,
 'is': 12,
 'live': 13,
 'of': 14,
 'paris': 15,
 'professor': 16,
 'stanford': 17,
 'taiwan': 18,
 'the': 19,
 'to': 20,
 'turkey': 21,
 'we': 22}

In [9]:
# We are ready to convert our training sentences into a sequence of indices corresponding to each token.

In [10]:
# Given a sentence of tokens, return the corresponding indices
# def convert_token_to_indices(sentences, word_to_ix):
#     indices = []
#     for token in sentences:
#         if token in word_into_ix:
#             index = word_to_ix[token]
#         else:
#             index = word_into_ix["<unk>"]
#         indices.append(index)
#     return indices
def convert_token_to_indices(sentence, word_to_ix):
    return [word_into_ix.get(token, word_into_ix["<unk>"])for token in sentence]

example_sentence = ["we", "always", "come", "to", "kuwait"]
example_indices = convert_token_to_indices(example_sentence, word_into_ix)
restored_example = [ix_into_words[ind] for ind in example_indices]

print(example_indices)
print(f"Original sentence is: {example_sentence}")
print(f"Going from words to indices: {example_indices}")
print(f"Going from indices to words: {restored_example}")

[22, 2, 6, 20, 1]
Original sentence is: ['we', 'always', 'come', 'to', 'kuwait']
Going from words to indices: [22, 2, 6, 20, 1]
Going from indices to words: ['we', 'always', 'come', 'to', '<unk>']


In [16]:
example_padded_indices = [convert_token_to_indices(s, word_into_ix) for s in train_sentences]
example_padded_indices

[[22, 2, 6, 20, 15],
 [19, 16, 12, 8, 4],
 [10, 13, 11, 17],
 [9, 7, 8, 18],
 [19, 5, 14, 21, 12, 3]]

In [12]:
embedding_dim = 5
embeds = nn.Embedding(len(vocabulary), embedding_dim)
list(embeds.parameters())

[Parameter containing:
 tensor([[-1.2748,  0.8922,  0.3010,  0.3080,  0.5173],
         [ 0.1768, -1.2304, -0.1921,  0.7793,  0.7772],
         [-0.3091,  1.5729,  0.2039,  0.5740,  0.5542],
         [-1.5117, -0.1493, -0.1691, -1.7332, -0.6309],
         [-0.9219, -1.3908, -0.6302,  0.3732, -1.7481],
         [-0.8977, -0.7104, -1.0385, -1.3624,  0.8264],
         [ 1.2901,  0.3173,  0.6260, -1.3747, -1.2978],
         [ 0.8592,  0.2615,  1.0472,  0.5182,  0.9857],
         [ 0.3388,  0.2828, -1.4897, -0.2431, -0.1406],
         [ 2.3407, -0.4130, -0.2359,  1.1005,  0.7202],
         [-0.5725,  0.1554,  1.0098, -0.2164,  0.8102],
         [-0.0843,  0.8028,  0.2297,  0.4454,  0.0695],
         [-0.6191, -0.7843, -0.2285, -1.6470,  0.5666],
         [-0.9508, -1.3548,  0.8492, -0.5451, -2.3403],
         [-0.5199, -1.2582, -0.4965,  0.3955,  1.6974],
         [ 1.6818, -1.1755, -0.3067, -0.8472, -1.1329],
         [-0.3929, -0.1059,  0.7593, -1.0430, -0.8098],
         [ 0.3838, -1.299

In [13]:
#we want to get the lookup tensor for the word Paris
index = word_into_ix.get('paris')
index_tensor = torch.tensor(index, dtype=torch.long)
paris_embed = embeds(index_tensor)
paris_embed


tensor([ 1.6818, -1.1755, -0.3067, -0.8472, -1.1329],
       grad_fn=<EmbeddingBackward0>)

In [28]:
from torch.utils.data import DataLoader
from functools import partial
def custom_collate_fn(batch, window_size,word_into_ix):
    # we are going to break out our batch into examples 'x' and labels 'y'
    # and then turn them into tensors because nn.utiles.rnn.pad_sequence
    # expects tensors as inputs
    x, y =zip(*batch)
    # we have already designed a function for padding but we're just gonna bring it here as well to have everything in one place
    def pad_window(sentence, window_size, pad_token="<pad>"):
        window = [pad_token] * window_size
        return window + sentence + window

    # now we pad our examples
    x = [pad_window(s, window_size=window_size) for s in x]

    # now we need to turn words in our training examples to indices.
    def convert_token_to_indices(sentence, word_to_ix):
        return [word_to_ix.get(token, word_to_ix['<unk>']) for token in sentence]

    # now we convert the examples themselves into indices
    x = [convert_token_to_indices(s, word_into_ix) for s in x]

    # now we pad all of our examples so they are all the same length
    # because our matrix operations are impossible if they are not the same length
    pad_token_ix = word_into_ix["<pad>"]

    # pad sequence expects tensors so we make x into a tensor
    x = [torch.LongTensor(x_i) for x_i in x]
    x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)

    # We will also pad the labels. Before we do that, we record the number
    # of labels so that we know how many words existed in each example
    lengths = [len(label) for label in y]
    lengths = torch.LongTensor(lengths)

    y = [torch.LongTensor(y_i) for y_i in y]
    y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)


    #we are now ready to return our variables. The order we return our variables
    # here will match the oder we read them in our training loop
    return x_padded, y_padded, lengths
    #this order is super important


In [29]:
# Params to be passed to the Dataloader
data = list(zip(train_sentences, train_labels))
batch_size = 2
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_into_ix=word_into_ix)

# Now we instantiate the Dataloader
loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

#Go through one loop
counter = 0
for batched_x, batched_y, batched_lengths in loader:
    print(f'Iteration {counter}')
    print('Batched Input:')
    print(batched_x)
    print('Batched Labels:')
    print(batched_y)
    print('Batched Lengths:')
    print(batched_lengths)
    print('')
    counter += 1

Iteration 0
Batched Input:
tensor([[ 0,  0,  9,  7,  8, 18,  0,  0,  0,  0],
        [ 0,  0, 19,  5, 14, 21, 12,  3,  0,  0]])
Batched Labels:
tensor([[0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 1]])
Batched Lengths:
tensor([4, 6])

Iteration 1
Batched Input:
tensor([[ 0,  0, 19, 16, 12,  8,  4,  0,  0],
        [ 0,  0, 10, 13, 11, 17,  0,  0,  0]])
Batched Labels:
tensor([[0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0]])
Batched Lengths:
tensor([5, 4])

Iteration 2
Batched Input:
tensor([[ 0,  0, 22,  2,  6, 20, 15,  0,  0]])
Batched Labels:
tensor([[0, 0, 0, 0, 1]])
Batched Lengths:
tensor([5])

